## imports

In [1]:
from pandas_datareader import data as pdr
import pandas as pd
import datetime
from bokeh.plotting import figure, show, output_file

## get dataframe

In [208]:
end = datetime.datetime(2016,3,9)
start = end - pd.DateOffset(months=4)
# start = datetime.date.today() - pd.DateOffset(days=10)
# end = datetime.date.today()
print(start, end)
goog = pdr.DataReader(name="GOOG", data_source="yahoo", start=start, end=end)
print(goog)

2015-11-09 00:00:00 2016-03-09 00:00:00
                  High         Low        Open       Close   Volume  \
Date                                                                  
2015-11-09  734.710022  719.429993  730.200012  724.890015  2069800   
2015-11-10  730.590027  718.500000  724.400024  728.320007  1608000   
2015-11-11  741.000000  730.229980  732.460022  735.400024  1366400   
2015-11-12  737.799988  728.645020  731.000000  731.229980  1837200   
2015-11-13  731.150024  716.729980  729.169983  717.000000  2075500   
...                ...         ...         ...         ...      ...   
2016-03-04  716.489990  706.020020  714.989990  710.890015  1972100   
2016-03-07  708.091003  686.900024  706.900024  695.159973  2985100   
2016-03-08  703.789978  685.340027  688.590027  693.969971  2076300   
2016-03-09  705.679993  694.000000  698.469971  705.239990  1421500   
2016-03-10  716.440002  703.359985  708.119995  712.820007  2833500   

             Adj Close  
Date       

## add status col to df

In [209]:
def stat(cl, op):
    if cl > op:
        value = "inc"
    elif op > cl:
        value = "dec"
    else:
        value = "eq"
    return value
goog["Status"] = [stat(c, o) for c, o in zip(goog.Close, goog.Open)]

## add mean and range cols to df

In [210]:
goog["Mean"] = (goog.Open + goog.Close)/2
goog["Range"] = (goog.Open - goog.Close)

## plot params

In [211]:
# get datestamps
plt_gr = goog.index[goog.Status == 'inc']
plt_rd = goog.index[goog.Status == 'dec']

# get glyph centers (mean)
plt_gr_y = goog.Mean[goog.Status == 'inc']
plt_rd_y = goog.Mean[goog.Status == 'dec']

# set width to 1/2 of x interval
plt_wt = (goog.index[1] - goog.index[0]) / 2

# get glyph heights (range)
plt_gr_h = goog.Range[goog.Status == 'inc']
plt_rd_h = goog.Range[goog.Status == 'dec']

gn = '#99ff99'
rd = '#ffb399'

## plot setup

In [212]:
plt = figure(x_axis_type='datetime', width=1000, height=300, sizing_mode='scale_width')
plt.title.text="Candlestick Chart"
plt.grid.grid_line_alpha = 0.75

# add range line segments
plt.segment(goog.index, goog.High, goog.index, goog.Low, color='black')

# plots for gains
plt.rect(x=plt_gr, y=plt_gr_y, width=plt_wt, height=abs(plt_gr_h), fill_color=gn, line_color='black')
# plots for losses
plt.rect(x=plt_rd, y=plt_rd_y, width=plt_wt, height=abs(plt_rd_h), fill_color=rd, line_color='black')

GlyphRenderer(id='13673', ...)

## show plot

In [213]:
output_file('cs.html')
show(plt)